In [1]:
import pandas as pd
import numpy as np
import re

#prefix = "/content/drive/MyDrive/Colab Notebooks/data_engineering/lab_02/"
#prefix="project02/"
prefix=""

# test_df = pd.read_csv("another_test_df.csv", delimiter=";")
# test_df

In [2]:
#TASK 1
#input_path = f"{prefix}proj2_data – comma float.csv"
input_path = f"{prefix}proj2_data.csv"

def is_at_least_one_float_col(df):
  for col_type in df.dtypes:
    if ("float" in str(col_type)):
      return True
  return False

#df read csv: decimal= str (length 1), default ‘.’

  #delim | and float delim "."
delim = "|"
df = pd.read_csv(input_path, delimiter=delim)
#print("|", len(df.columns))
has_float = is_at_least_one_float_col(df)
  #delim | and float delim ","
if(not has_float): #no float col was found - delim is not ok
  df = pd.read_csv(input_path, delimiter=delim, decimal=",")

  #delim ; and float delim "."
if(len(df.columns)==1):
  delim = ";"
  df = pd.read_csv(input_path, delimiter=delim)
  #print(";",len(df.columns))
  has_float = is_at_least_one_float_col(df)
    #delim ; and float delim ","
  if(not has_float): #no float col was found - delim is not ok
    df = pd.read_csv(input_path, delimiter=delim, decimal=",")

  if(len(df.columns)==1): #if we get to third case, it means no number will be separated by , as it is col separator
    delim = ","
    df = pd.read_csv(input_path, delimiter=delim)
    #print(",",len(df.columns))

df.to_pickle(f"{prefix}proj2_ex01.pkl")
init_df = df.copy()


init_df


,full_name,field,language,code,task_1,task_2,task_3,tasks_avg,task_grade,jury_score,final_grade
0,Rowan Harrington,drones,python,wej,3.1,2.0,4.4,3.166667,dostateczny,"3,5 pts",dostateczny
1,Nash Wyatt,racing,java,sfe,4.2,2.0,2.0,2.733333,bardzo dobry,5 p,mierny
2,Jadiel Ramirez,media,cplusplus,vaw,4.0,4.9,3.0,3.966667,dobry,3.5,mierny
3,Makaila Atkins,racing,swift,ugt,4.1,5.0,4.8,4.633333,dobry,2,dostateczny
4,Melanie Fuller,racing,python,owb,2.7,2.0,2.0,2.233333,bardzo dobry,pts 2,mierny
5,Layla Woodard,drones,python,wuf,2.8,2.0,2.0,2.266667,bardzo dobry,"3,5p",dostateczny
6,Edwin Hale,media,swift,ceq,4.9,3.0,3.4,3.766667,dostateczny,-1 (dnf),niedostateczny
7,Travis Rocha,robotics,cplusplus,eir,5.0,4.9,5.0,4.966667,dostateczny,0,niedostateczny
8,Ricky Howell,media,java,vos,2.1,2.0,2.0,2.033333,dobry,waiting,niedostateczny
9,Addyson Brennan,drones,java,cov,4.4,3.7,4.0,4.033333,bardzo dobry,-,niedostateczny


In [3]:
#TASK 2
scale = []
scale_dict = {}

# counter = 1
# with open(f"{prefix}proj2_scale.txt") as f:
#     line = f.readline().strip()
#     while line:
#         scale.append(line)
#         scale_dict[line] = counter
#         counter +=1
#         line = f.readline().strip()


with open(f"{prefix}proj2_scale.txt") as f:
    content = f.read()  


scale = content.split('\n')
for i,val in enumerate(scale):
  scale_dict[val] = i+1
#print(scale_dict)


scale_df = init_df.copy()
categorical_columns = set()

def replace_if_scale(row):
    for col in row.index:
        for key, val in scale_dict.items():
            if key == str(row[col]):
                row[col] = val
                categorical_columns.add(col)
    return row

scale_df = scale_df.apply(replace_if_scale, axis=1) #apply goes through rows 0/columns 1
scale_df.to_pickle(f"{prefix}proj2_ex02.pkl")
scale_df



,full_name,field,language,code,task_1,task_2,task_3,tasks_avg,task_grade,jury_score,final_grade
0,Rowan Harrington,drones,3,wej,3.1,2.0,4.4,3.166667,dostateczny,"3,5 pts",dostateczny
1,Nash Wyatt,racing,1,sfe,4.2,2.0,2.0,2.733333,bardzo dobry,5 p,mierny
2,Jadiel Ramirez,media,2,vaw,4.0,4.9,3.0,3.966667,dobry,3.5,mierny
3,Makaila Atkins,racing,4,ugt,4.1,5.0,4.8,4.633333,dobry,2,dostateczny
4,Melanie Fuller,racing,3,owb,2.7,2.0,2.0,2.233333,bardzo dobry,pts 2,mierny
5,Layla Woodard,drones,3,wuf,2.8,2.0,2.0,2.266667,bardzo dobry,"3,5p",dostateczny
6,Edwin Hale,media,4,ceq,4.9,3.0,3.4,3.766667,dostateczny,-1 (dnf),niedostateczny
7,Travis Rocha,robotics,2,eir,5.0,4.9,5.0,4.966667,dostateczny,0,niedostateczny
8,Ricky Howell,media,1,vos,2.1,2.0,2.0,2.033333,dobry,waiting,niedostateczny
9,Addyson Brennan,drones,1,cov,4.4,3.7,4.0,4.033333,bardzo dobry,-,niedostateczny


In [4]:
#TASK 3
cat_df = init_df.copy()

for col_name in categorical_columns:
  cat_df[col_name] = cat_df[col_name].astype("category")
  cat_df[col_name] = cat_df[col_name].cat.set_categories(scale)

cat_df.to_pickle(f"{prefix}proj2_ex03.pkl")

#print(cat_df['language'].cat.categories)
cat_df.dtypes


full_name        object
field            object
language       category
code             object
task_1          float64
task_2          float64
task_3          float64
tasks_avg       float64
task_grade       object
jury_score       object
final_grade      object
dtype: object

In [5]:
#TASK 4

extracted_num_df = init_df.copy()



new_df = pd.DataFrame()

# #-? optional -  \d+ one or more digits  (?:[.,]\d+)?  optional dot or comma + digits
# def extract_number (input):
#   pattern = r'-?\d+(?:[.,]\d+)?'
#   match = re.search(pattern, str(input))
#   if match:
#       #print(type(match.group()))
#       return match.group()
#   return None

def extract_number(input):
    res = ''
    caught_some_number = False

    for i,char in enumerate(input):
        if char.isdigit() or char in ['.', ',', '-']:
            if char=="-":
                if i!=(len(input)-1) and input[i+1].isdigit():
                    res+=char
            elif char=="." or char==",":
                if i!=(0) and i!=(len(input)-1) and input[i-1].isdigit() and input[i+1].isdigit():
                    res+="."
            else:
                res += char
                caught_some_number = True
        elif not char.isdigit() and char not in ['.', ',', '-'] and caught_some_number:
            break
            
    if any(char_check.isdigit() for char_check in res):
        try:
            return float(res)
        except ValueError:
            return None
    return None

for i,col in extracted_num_df.items(): #we go column by column
  if col.dtype=="object":
    was_extracted=False
    new_col = []
    for val in col:
      extracetd = extract_number(val)
      if extracetd is not None:
        was_extracted=True
      new_col.append(extracetd)
    if was_extracted:
      new_df[col.name] = new_col


new_df.to_pickle(f"{prefix}proj2_ex04.pkl")
new_df


,jury_score
0,3.5
1,5.0
2,3.5
3,2.0
4,2.0
5,3.5
6,-1.0
7,0.0
8,NaN
9,NaN


In [6]:
#TASK 5

one_hot_df = init_df.copy()

to_be_encoded = []

for i,col in one_hot_df.items(): #we go column by column
  if col.dtype=="object":
    vals = set()
    good_vals = True
    for val in col:
      if val is None or not val.isalpha() or not val.islower() or val in scale:
          good_vals=False
      else:
        vals.add(val)
    #print(col.name, ":", vals, "is good:", good_vals)
    if len(vals)<=10 and good_vals:
      to_be_encoded.append(col.name)

print(to_be_encoded)
ord=0
for col_name in to_be_encoded:
  ord+=1
  one_hot_df_col = pd.get_dummies(one_hot_df[col_name])
  one_hot_df_col.to_pickle(f"{prefix}proj2_ex05_{ord}.pkl")
  print(one_hot_df_col)





['field', 'final_grade']
    drones  media  racing  robotics
0     True  False   False     False
1    False  False    True     False
2    False   True   False     False
3    False  False    True     False
4    False  False    True     False
5     True  False   False     False
6    False   True   False     False
7    False  False   False      True
8    False   True   False     False
9     True  False   False     False
10    True  False   False     False
11   False  False    True     False
    dostateczny  mierny  niedostateczny
0          True   False           False
1         False    True           False
2         False    True           False
3          True   False           False
4         False    True           False
5          True   False           False
6         False   False            True
7         False   False            True
8         False   False            True
9         False   False            True
10         True   False           False
11        False    True    